<a href="https://colab.research.google.com/github/Prkhar05/Tiny_Text2SQL/blob/main/TinyTrain_Gemma_it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and Imports

In [ ]:
!pip install -U -q accelerate peft bitsandbytes git+https://github.com/huggingface/transformers trl datasets transformers[sentencepiece]

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -U -q datasets==2.16.0

In [ ]:
import torch
from datasets import load_dataset,Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftConfig, PeftModel,get_peft_model
from trl import SFTTrainer


In [ ]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
# secret_wandb = user_secrets.get_secret("wandb")

In [ ]:
# !huggingface-cli login --token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
use_auth_token=True

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key="0d08dc6eea8c357e2d10ab5ee7e85b4ac6cf4254")

wandb: Currently logged in as: prkharmishra05. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Setting the Fine-tuning Parameters

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Meta-Llama-3-8B"

# # The instruction dataset to use
dataset_name = "PipableAI/pip-txt-to-sql-spider-bird-dataset"

# Fine-tuned model name
new_model = "sft-Meta-Llama-3-8B"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 64

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = f"./{new_model}"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = False

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 5e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.5

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 500

# Log every X updates steps
logging_steps = 100

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 1800

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Fine-tuning and dataset loading

In [ ]:
dataset = load_dataset(dataset_name,split='train')
def prompt_pip_sql(d):
  prompt = f"""<schema>{d['schema']}</schema>
    <question>{d['question']}</question>
    <sql>{d['query']}</sql>"""
  return prompt

texts=[prompt_pip_sql(d) for d in dataset]
data_proc = Dataset.from_dict({"text" : texts})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# #Importing the dataset
# from datasets import load_dataset
# dataset = load_dataset(dataset_name, split="small_train")
# eval_dataset = load_dataset(dataset_name, split="small_validation")

# dataset["prompt"][100]

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
# from accelerate import PartialState

# model = AutoModelForCausalLM.from_pretrained(model_name,
#   low_cpu_mem_usage=True,
#   torch_dtype=torch.float16,
#   load_in_8bit=True,
#   device_map={"": PartialState().process_index},)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [ ]:
# print(model.transformer)

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )
print_trainable_parameters(model)
# model.named_parameters()

trainable params: 1050939392 || all params: 4540600320 || trainable%: 23.15


In [ ]:
# target_modules=["q_proj", "v_proj"]
# target_modules=["q", "v"]
# target_modules=["query_key_value"]
# target_modules=None
# target_modules="all-linear"

# def find_all_linear_names(model):
#     lora_module_names = set()
#     for name, module in model.named_modules():
#         if isinstance(module, bnb.nn.Linear4bit):
#             names = name.split(".")
#             lora_module_names.add(names[0] if len(names) == 1 else names[-1])

#     if "lm_head" in lora_module_names:  # needed for 16-bit
#         lora_module_names.remove("lm_head")
#     return list(lora_module_names)

# def get_specific_layer_names(model):
#     layer_names = []
#         for name, module in model.named_modules():
#         if isinstance(module, (torch.nn.Linear, torch.nn.Embedding, torch.nn.Conv2d, Conv1D)):
#             layer_names.append('.'.join(name.split('.')[4:]).split('.')[0])

#     return layer_names

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
            "q_proj",
            "v_proj",
            "k_proj",
            "out_proj",
            "fc_in",
            "fc_out",
            "wte",
        ],
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
m=get_peft_model(model,peft_config)
m.print_trainable_parameters()

trainable params: 37,748,736 || all params: 8,068,009,984 || trainable%: 0.4678816222942344


In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
#     push_to_hub = True,
    )

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=data_proc,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/16428 [00:00<?, ? examples/s]

In [ ]:
trainer.is_model_parallel

False

In [ ]:
trainer.train()
trainer.model.save_pretrained(new_model)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/run

In [ ]:
# adapter_config_path = '/content/sft-pip-sql'
# model.config.to_json_file(adapter_config_path)
# model.push_to_hub("new_model")


In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
# wandb.finish()
model.config.use_cache = True
model.eval()

In [ ]:
model.push_to_hub("sft-gemma-1.1-2b-it-v3")